# Zika virus (データの正確な表現が目的)

ジカウイルスの発生は、主にヒトスジシマカ（Ae。aegyptiおよびAe。albopictus）によって伝染する感染症であり、2015年半ばから南北アメリカおよび太平洋全体に広がっています。 1947年にウガンダで最初に隔離されましたが、以前の研究が不足しているため、流行が広がり続けているため、その壊滅的な影響を迅速に理解することが科学界に挑戦しています。
<br>
データこのデータセットは、現在進行中のジカの流行に関連して公開されているデータを共有しています。これは、公衆衛生対策に従事する科学コミュニティへのリソースとして提供されています。ここで提供されるデータは公式ではなく、暫定的で網羅的ではないと考えるべきです。

レポートのデータは、レポートの遅延または分類の変更を反映して、時間とともに変化する場合があります。また、ここで共有される機械可読ファイルでは、報告されたデータの正確な表現が目的ですが、その正確性は保証されていません。これらのデータを使用する前に、CDC Zika流行のGitHubリポジトリに関する詳細情報とともに、可能な限り提供される元のレポートとソースを確認することをお勧めします。

データセットには次のフィールドが含まれます。

1. report_date-レポートの日付は、レポートが公開された日付です。日付は標準ISO形式（YYYY-MM-DD）で指定する必要があります。

1. 場所-場所は、国の地名データベースで指定された特定の名前に続く各観測に対して指定されます。これは、以下にリストされているように、「location_type」を持つ任意の場所です。市、州、国など。次の形式で最大3つの階層レベルで指定する必要があります。[国]-[州/県]-[国/市区町村/市]。常に国名で始まります。データが特定の都市のものである場合、例えばサルバドール、指定する必要があります：ブラジルバイアサルバドール。 

1. location_type-以下を示すロケーションコードが含まれます：都市、地区、自治体、郡、州、県、または国。追加の「location_type」が必要な場合は、課題を開いて新しい「location_type」を作成します。 

1. data_field-データフィールドは、行に表示されるデータの簡単な説明であり、元のレポートで定義された特定の定義に関連しています。 

1. data_field_code-このコードは、国のデータガイドで定義されています。 2文字の国コード（ISO-3166 alpha-2、リスト）が含まれ、その後に特定のレポートタイプとデータタイプに対応する4桁の数字が続きます。 

1. time_period-オプション。データが特定の期間、たとえば疫学的な週に関連する場合、その数と「time_period_type」の期間のタイプをここに示す必要があります。そうでない場合はNAになります。 

1. time_period_type-'time_period'が指定されている場合のみ必要です。タイプは、国のデータガイドでも指定されます。それ以外の場合はNAになります。 

1. value-特定の「report_date」、「location」、「data_field」、および必要に応じて「time_period」に示される観測。 

1. unit-'data_field'の測定単位。これは、国固有のデータガイドで説明されている「data_field」ユニットオプションに準拠する必要があります。データが有用な場合は、このデータセットと元のデータソースを参照して、データ共有をサポートしてください。 

GitHubからZikaプロジェクトへの貢献に興味がある場合は、こちらで詳細を読むことができます。 Zikaウイルス構造のソースは、ここから入手できます。

## データ分析
必要になるライブラリ呼ぶ

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

from collections import Counter

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


# データ読み込み

In [3]:
train_df = pd.read_csv('~/Dropbox/@00_input/Zika_Virus/cdc_zika.csv')

データフレームのカラムを取得

In [4]:
print(train_df.columns.values)
#print(test_df.columns.values)

['report_date' 'location' 'location_type' 'data_field' 'data_field_code'
 'time_period' 'time_period_type' 'value' 'unit']


In [5]:
print("train data size", train_df.shape)
#print("train data size", test_df.shape)

train data size (107619, 9)


In [6]:
train_df.head()

,report_date,location,location_type,data_field,data_field_code,time_period,time_period_type,value,unit
0,2016-03-19,Argentina-Buenos_Aires,province,cumulative_confirmed_local_cases,AR0001,NaN,NaN,0,cases
1,2016-03-19,Argentina-Buenos_Aires,province,cumulative_probable_local_cases,AR0002,NaN,NaN,0,cases
2,2016-03-19,Argentina-Buenos_Aires,province,cumulative_confirmed_imported_cases,AR0003,NaN,NaN,2,cases
3,2016-03-19,Argentina-Buenos_Aires,province,cumulative_probable_imported_cases,AR0004,NaN,NaN,1,cases
4,2016-03-19,Argentina-Buenos_Aires,province,cumulative_cases_under_study,AR0005,NaN,NaN,127,cases


null チェック

In [7]:
train_df.info()
print('_'*40)
# test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107619 entries, 0 to 107618
Data columns (total 9 columns):
report_date         107612 non-null object
location            107612 non-null object
location_type       107612 non-null object
data_field          107612 non-null object
data_field_code     107612 non-null object
time_period         0 non-null float64
time_period_type    0 non-null float64
value               107481 non-null object
unit                107612 non-null object
dtypes: float64(2), object(7)
memory usage: 7.4+ MB
________________________________________


## データの確認

## location
1674種類
<br>
国の地名データベースで指定された特定の名前に続く各観測に対して指定されます。これは、以下にリストされているように、「location_type」を持つ任意の場所です。市、州、国など。次の形式で最大3つの階層レベルで指定する必要があります。[国]-[州/県]-[国/市区町村/市]。常に国名で始まります。データが特定の都市のものである場合、例えばサルバドール、指定する必要があります：ブラジルバイアサルバドール。 location_type-以下を示すロケーションコードが含まれます：都市、地区、自治体、郡、州、県、または国。追加の「location_type」が必要な場合は、課題を開いて新しい「location_type」を作成します。

In [9]:
mycounter = Counter(train_df.location)
n = 1
for word in mycounter.most_common():
    print(word, n)
    n = n +1

('United_States_Virgin_Islands', 413) 1
('El_Salvador', 305) 2
('Puerto_Rico', 260) 3
('Ecuador', 225) 4
('Dominican_Republic', 201) 5
('Dominican_Republic-Distrito_Nacional', 175) 6
('Dominican_Republic-Santo_Domingo', 172) 7
('Brazil-Alagoas', 171) 8
('Brazil-Bahia', 171) 9
('Brazil-Ceara', 171) 10
('Brazil-Goias', 171) 11
('Brazil-Paraiba', 171) 12
('Brazil-Pernambuco', 171) 13
('Brazil-Piaui', 171) 14
('Brazil-Rio_Grande_do_Norte', 171) 15
('Brazil-Acre', 170) 16
('Brazil-Maranhao', 170) 17
('Brazil-Mato_Grosso', 170) 18
('Brazil-Minas_Gerais', 170) 19
('Brazil-Parana', 170) 20
('Brazil-Sergipe', 170) 21
('Brazil-Sao_Paulo', 170) 22
('Dominican_Republic-Independencia', 170) 23
('Dominican_Republic-Puerto_Plata', 170) 24
('Dominican_Republic-Azua', 170) 25
('Dominican_Republic-Barahona', 169) 26
('Dominican_Republic-Santiago', 167) 27
('Dominican_Republic-San_Cristobal', 167) 28
('Dominican_Republic-San_Pedro_de_Macoris', 167) 29
('Dominican_Republic-Espaillat', 167) 30
('Dominican_

('Centro-Oeste', 7) 1500
('Brazil', 7) 1501
(nan, 7) 1502
('Guatemala-Ixcan', 7) 1503
('Guatemala-Solola', 7) 1504
('Ecuador-Bolivar', 6) 1505
('Ecuador-Guayas-General_Antonio_Elizalde', 6) 1506
('Ecuador-Loja', 6) 1507
('Ecuador-Guayas-General_Antonio_Elizalde ', 6) 1508
('United_States-Rhode_Island', 6) 1509
('United_States-South_Carolina', 6) 1510
('United_States-Vermont', 6) 1511
('United_States-Wisconsin', 6) 1512
('Ecuador-Canar', 5) 1513
('Ecuador-Carchi', 5) 1514
('Ecuador-Esmeraldas-Quininde', 5) 1515
('El_Salvador-Honduras', 5) 1516
('Dominican_Republic-Other', 4) 1517
('Ecuador-Napo', 4) 1518
('Ecuador-Imbabura', 4) 1519
('Ecuador-Manabi-Jaramijo', 4) 1520
('Ecuador-Manabi-Sucre', 4) 1521
('United_States-Pennsylvania††', 4) 1522
('Ecuador-Manabi-Jipijapa', 3) 1523
('Ecuador-Manabi-Santa_Ana', 3) 1524
('Nicaragua-Managua-Distrito_IV', 3) 1525
('Ecuador-Los_Rios-Jaramijo', 2) 1526
('Ecuador-El_Oro-Santa_Rosa', 2) 1527
('Ecuador-Esmeraldas-San_Lorenzo', 2) 1528
('Ecuador-Manabi

## location_type
12種類
<br>
都市、地区、自治体、郡、州、県、または国。追加の「location_type」が必要な場合は、課題を開いて新しい「location_type」を作成します。

In [10]:
mycounter = Counter(train_df.location_type)
n = 1
for word in mycounter.most_common():
    print(word, n)
    n = n +1

('municipality', 87557) 1
('state', 8462) 2
('province', 7583) 3
('county', 1613) 4
('country', 866) 5
('territory', 777) 6
('department', 600) 7
('Department', 95) 8
('region', 35) 9
('city', 14) 10
('district', 10) 11
(nan, 7) 12


## data_field
149種類
<br>
行に表示されるデータの簡単な説明であり、元のレポートで定義された特定の定義に関連しています。

In [12]:
mycounter = Counter(train_df.data_field)
n = 1
for word in mycounter.most_common():
    print(word, n)
    n = n +1

('zika_confirmed_laboratory', 28963) 1
('zika_suspected', 28963) 2
('zika_suspected_clinic', 16170) 3
('zika_confirmed_clinic', 12793) 4
('yearly_reported_travel_cases', 1035) 5
('weekly_zika_confirmed', 960) 6
('yearly_cumulative_female', 960) 7
('yearly_cumulative_male', 960) 8
('zika_reported_travel', 709) 9
('zika_reported_local', 709) 10
('microcephaly_under_investigation', 567) 11
('microcephaly_confirmed', 567) 12
('microcephaly_not', 567) 13
('municipality_microcephaly', 553) 14
('municipality_microcephaly_suspected', 540) 15
('total_zika_new_suspected', 480) 16
('microcephaly_fatal_under_investigation', 421) 17
('microcephaly_fatal_confirmed', 421) 18
('microcephaly_fatal_not', 421) 19
('cumulative_suspected_total', 413) 20
('gbs_reported', 381) 21
('zika_suspected_cumulative', 380) 22
('zika_suspected_pregnant_cumulative', 380) 23
('zika_suspected_pregnant', 378) 24
('zika_confirmed_pcr_cumulative', 376) 25
('gbs_confirmed_cumulative', 374) 26
('gbs_reported_cumulative', 374)

## data_field_code
154種類
<br>
国のデータガイドで定義されています。 2文字の国コード（ISO-3166 alpha-2、リスト）が含まれ、その後に特定のレポートタイプとデータタイプに対応する4桁の数字が続きます。

In [13]:
mycounter = Counter(train_df.data_field_code)
n = 1
for word in mycounter.most_common():
    print(word, n)
    n = n +1

('CO0001', 28963) 1
('CO0003', 28963) 2
('CO0004', 16170) 3
('CO0002', 12793) 4
('US0003', 1035) 5
('MX0001', 960) 6
('MX0002', 960) 7
('MX0003', 960) 8
('US0001', 709) 9
('US0002', 709) 10
('BR0001', 567) 11
('BR0002', 567) 12
('BR0003', 567) 13
('BR0004', 553) 14
('BR0008', 540) 15
('DO0001', 426) 16
('BR0005', 421) 17
('BR0006', 421) 18
('BR0007', 421) 19
('SV0010', 413) 20
('DO0010', 381) 21
('DO0005', 380) 22
('DO0007', 380) 23
('DO0008', 378) 24
('DO0006', 376) 25
('DO0020', 374) 26
('DO0014', 374) 27
('DO0023', 374) 28
('DO0021', 374) 29
('DO0022', 374) 30
('DO0017', 374) 31
('DO0024', 374) 32
('DO0025', 374) 33
('DO0026', 374) 34
('AR0001', 336) 35
('AR0002', 336) 36
('AR0003', 336) 37
('AR0004', 336) 38
('AR0005', 336) 39
('AR0006', 336) 40
('SV0011', 334) 41
('EC0003', 287) 42
('BR0011', 231) 43
('GT0007', 178) 44
('EC0001', 147) 45
('GT0001', 106) 46
('GT0008', 101) 47
('EC0004', 86) 48
('VI0001', 80) 49
('GT0010', 79) 50
('EC0005', 70) 51
('PA0001', 61) 52
('PA0002', 60) 53

## value
1770週類
<br>
特定の「report_date」、「location」、「data_field」、および必要に応じて「time_period」に示される観測。

In [14]:
mycounter = Counter(train_df.value)
n = 1
for word in mycounter.most_common():
    print(word, n)
    n = n +1

('0', 45134) 1
(0.0, 25071) 2
('1', 4872) 3
(1.0, 3551) 4
('2', 2386) 5
(2.0, 1939) 6
('3', 1411) 7
(3.0, 1107) 8
(4.0, 935) 9
('4', 837) 10
('5', 758) 11
('6', 645) 12
(6.0, 584) 13
(5.0, 582) 14
(7.0, 498) 15
('7', 446) 16
(9.0, 414) 17
('9', 384) 18
('8', 376) 19
('10', 355) 20
(10.0, 340) 21
(8.0, 327) 22
(11.0, 284) 23
('11', 283) 24
('13', 279) 25
(14.0, 250) 26
('14', 249) 27
(12.0, 239) 28
('15', 235) 29
('12', 229) 30
(15.0, 218) 31
(13.0, 213) 32
(17.0, 193) 33
('16', 192) 34
('17', 186) 35
(16.0, 177) 36
('19', 176) 37
('20', 171) 38
('18', 156) 39
(22.0, 156) 40
('21', 143) 41
(21.0, 142) 42
(19.0, 138) 43
('22', 135) 44
('23', 121) 45
(18.0, 119) 46
(nan, 118) 47
('24', 118) 48
('36', 109) 49
(20.0, 109) 50
(25.0, 105) 51
(24.0, 104) 52
('26', 94) 53
(23.0, 94) 54
('28', 93) 55
(29.0, 92) 56
('30', 89) 57
('31', 89) 58
('29', 86) 59
('27', 85) 60
('33', 84) 61
('39', 84) 62
('25', 83) 63
(27.0, 82) 64
(30.0, 79) 65
('40', 75) 66
(26.0, 73) 67
('38', 70) 68
('35', 69) 69
('

(822.0, 1) 1487
(143.0, 1) 1488
(991.0, 1) 1489
(1090.0, 1) 1490
(466.0, 1) 1491
(360.0, 1) 1492
(556.0, 1) 1493
(659.0, 1) 1494
(1747.0, 1) 1495
(816.0, 1) 1496
(729.0, 1) 1497
(188.0, 1) 1498
(2096.0, 1) 1499
(558.0, 1) 1500
(967.0, 1) 1501
(948.0, 1) 1502
(2443.0, 1) 1503
(1171.0, 1) 1504
(1122.0, 1) 1505
(563.0, 1) 1506
(2731.0, 1) 1507
(1300.0, 1) 1508
(1180.0, 1) 1509
(241.0, 1) 1510
(2987.0, 1) 1511
(1406.0, 1) 1512
(1148.0, 1) 1513
(752.0, 1) 1514
(1525.0, 1) 1515
(1153.0, 1) 1516
(520.0, 1) 1517
(3477.0, 1) 1518
(774.0, 1) 1519
(1580.0, 1) 1520
(3771.0, 1) 1521
(827.0, 1) 1522
(172.0, 1) 1523
(1689.0, 1) 1524
(992.0, 1) 1525
(658.0, 1) 1526
(254.0, 1) 1527
(551.0, 1) 1528
(864.0, 1) 1529
(1113.0, 1) 1530
(394.0, 1) 1531
(450.0, 1) 1532
(1692.0, 1) 1533
(2773.0, 1) 1534
(728.0, 1) 1535
(560.0, 1) 1536
(438.0, 1) 1537
(2272.0, 1) 1538
(3836.0, 1) 1539
(990.0, 1) 1540
(979.0, 1) 1541
(747.0, 1) 1542
(397.0, 1) 1543
(492.0, 1) 1544
(605.0, 1) 1545
(1561.0, 1) 1546
(2474.0, 1) 1547

## unit
1770種類
<br>
これは、国固有のデータガイドで説明されている「data_field」ユニットオプションに準拠する必要があります。データが有用な場合は、このデータセットと元のデータソースを参照して、データ共有をサポートしてください。

In [17]:
mycounter = Counter(train_df.value)
n = 1
for word in mycounter.most_common():
    print(word, n)
    n = n +1

('0', 45134) 1
(0.0, 25071) 2
('1', 4872) 3
(1.0, 3551) 4
('2', 2386) 5
(2.0, 1939) 6
('3', 1411) 7
(3.0, 1107) 8
(4.0, 935) 9
('4', 837) 10
('5', 758) 11
('6', 645) 12
(6.0, 584) 13
(5.0, 582) 14
(7.0, 498) 15
('7', 446) 16
(9.0, 414) 17
('9', 384) 18
('8', 376) 19
('10', 355) 20
(10.0, 340) 21
(8.0, 327) 22
(11.0, 284) 23
('11', 283) 24
('13', 279) 25
(14.0, 250) 26
('14', 249) 27
(12.0, 239) 28
('15', 235) 29
('12', 229) 30
(15.0, 218) 31
(13.0, 213) 32
(17.0, 193) 33
('16', 192) 34
('17', 186) 35
(16.0, 177) 36
('19', 176) 37
('20', 171) 38
('18', 156) 39
(22.0, 156) 40
('21', 143) 41
(21.0, 142) 42
(19.0, 138) 43
('22', 135) 44
('23', 121) 45
(18.0, 119) 46
(nan, 118) 47
('24', 118) 48
('36', 109) 49
(20.0, 109) 50
(25.0, 105) 51
(24.0, 104) 52
('26', 94) 53
(23.0, 94) 54
('28', 93) 55
(29.0, 92) 56
('30', 89) 57
('31', 89) 58
('29', 86) 59
('27', 85) 60
('33', 84) 61
('39', 84) 62
('25', 83) 63
(27.0, 82) 64
(30.0, 79) 65
('40', 75) 66
(26.0, 73) 67
('38', 70) 68
('35', 69) 69
('

(665.0, 1) 1479
(435.0, 1) 1480
(1711.0, 1) 1481
(304.0, 1) 1482
(nan, 1) 1483
(305.0, 1) 1484
(573.0, 1) 1485
(670.0, 1) 1486
(822.0, 1) 1487
(143.0, 1) 1488
(991.0, 1) 1489
(1090.0, 1) 1490
(466.0, 1) 1491
(360.0, 1) 1492
(556.0, 1) 1493
(659.0, 1) 1494
(1747.0, 1) 1495
(816.0, 1) 1496
(729.0, 1) 1497
(188.0, 1) 1498
(2096.0, 1) 1499
(558.0, 1) 1500
(967.0, 1) 1501
(948.0, 1) 1502
(2443.0, 1) 1503
(1171.0, 1) 1504
(1122.0, 1) 1505
(563.0, 1) 1506
(2731.0, 1) 1507
(1300.0, 1) 1508
(1180.0, 1) 1509
(241.0, 1) 1510
(2987.0, 1) 1511
(1406.0, 1) 1512
(1148.0, 1) 1513
(752.0, 1) 1514
(1525.0, 1) 1515
(1153.0, 1) 1516
(520.0, 1) 1517
(3477.0, 1) 1518
(774.0, 1) 1519
(1580.0, 1) 1520
(3771.0, 1) 1521
(827.0, 1) 1522
(172.0, 1) 1523
(1689.0, 1) 1524
(992.0, 1) 1525
(658.0, 1) 1526
(254.0, 1) 1527
(551.0, 1) 1528
(864.0, 1) 1529
(1113.0, 1) 1530
(394.0, 1) 1531
(450.0, 1) 1532
(1692.0, 1) 1533
(2773.0, 1) 1534
(728.0, 1) 1535
(560.0, 1) 1536
(438.0, 1) 1537
(2272.0, 1) 1538
(3836.0, 1) 1539
(9

# データ表現方法検

- location :1674種類
- location_type:12種類
- data_field: 149種類 
- data_field_code: 154種類
-  value:1770週類
- unit:1770種類


In [18]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107619 entries, 0 to 107618
Data columns (total 9 columns):
report_date         107612 non-null object
location            107612 non-null object
location_type       107612 non-null object
data_field          107612 non-null object
data_field_code     107612 non-null object
time_period         0 non-null float64
time_period_type    0 non-null float64
value               107481 non-null object
unit                107612 non-null object
dtypes: float64(2), object(7)
memory usage: 7.4+ MB


In [19]:
print("train data size", train_df.shape)

train data size (107619, 9)
